In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import gym
import shap
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt

sys.path.append("..")

from models.DQN import DQNAgent
from models.PPO import PPOAgent
from models.DDPG import DDPGAgent
from mpl_toolkits import mplot3d
from core.visuals import *
from models.reward_functions import *
from tensorflow.keras.layers import *
from core.stochastic_similarity import similarity
from tensorflow.keras.optimizers import *


plt.style.use("ggplot")

In [ ]:
env = gym.make("LunarLander-v2")
dqn_agent = DQNAgent(env, Identity, RMSprop(1e-4, decay=0.99), model_name='lunarlander', batch_size=32, warmup=0, validation_logging=5,validation_episodes=5,layers=[128,128],activation='relu',epsilon_min=0.05,epsilon_decay=0.999,tau=0.01,reg=0.01,gamma=0.999)
dqn_agent.learn(1000)

In [ ]:
frames, observations, actions = run_model(env, 100, dqn_agent.critic.predict)

In [ ]:
env = gym.make("LunarLander-v2")
dqn_shap_agent = DQNAgent(env, dqn_shap, RMSprop(1e-4, decay=0.99), model_name='lunarlander', batch_size=32, warmup=0, validation_logging=5,validation_episodes=5,layers=[128,128],activation='relu',epsilon_min=0.05,epsilon_decay=0.999,tau=0.01,reg=0.01,gamma=0.999)
dqn_agent.learn(1000)

In [ ]:
for i in range(8):
    for j in range(8):
        plot = action_space_contour(env, i, j, dqn_agent.critic.batch_predict, discrete=True, figsize=(8, 6), dpi=150)
        plt.title("Shapley Reward Policy")
        plt.savefig(f"dqn_shap_plots/obs{i}{j}.png")
        plt.close();

In [ ]:
env_cont = gym.make("LunarLanderContinuous-v2")
ddpg_agent = DDPGAgent(env_cont, Identity, critic_optimizer=RMSprop(5e-3, decay=0.75),actor_optimizer=RMSprop(5e-3, decay=0.99),model_name='reloading',batch_size=256,warmup=0,actor_layers=[128,256],critic_layers=[128,256],actor_reg=0.01,critic_reg=0.01,activation=ELU,
    epsilon_min=0.01,tau=0.01,gamma=0.999,validation_logging=10,validation_episodes=5)
ddpg_agent.learn(1000)

In [ ]:
_ = run_model(env_cont, 10, ddpg_agent.actor.predict)